<a href="https://colab.research.google.com/gist/ricardoV94/c421ddacb3ba7a19ac46efa253ea466c/rosetta_stone_pymc_stan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A rosetta stone for PyMC and Stan

This notebook discusses how models can be translated between PyMC and Stan, highlightling syntax differences as well as general emphasis in how a model is constructed and used between the two libraries.

In [ ]:
%%capture
!pip install pystan

In [ ]:
import numpy as np

import stan
import nest_asyncio

from pymc import (
    Model,
    Data,
    Potential,
    CustomDist,
    Flat,
    Normal,
    Exponential,
    HalfNormal,
    observe,
    draw,
    sample_prior_predictive,
)
from pymc.distributions import transforms
from pymc import math

In [ ]:
nest_asyncio.apply()  # Needed for stan in a Jupyter/Colab environment

## A simple model

Let's define a very simple model in both libraries.

$$
\begin{align*}
x_i &\sim \text{Exponential}(1) \quad \text{for } i = 1, 2, 3 \\
\sigma_i &= x_i + 1 \\
y_i &\sim \text{Normal}(0, \sigma_i) \quad \text{for } i = 1, 2, 3 \\
\end{align*}
$$

In [ ]:
with Model() as pymc_model:
    x = Exponential("x", 1, shape=(3,))
    sigma = x + 1
    Normal("y", 0, sigma, observed=[1, 2, 3])

In [ ]:
stan_model_code = """
data {
  int< lower=0 > N;
  vector[N] y;
}

parameters {
  vector< lower=0 >[3] x;
}

model {
    x ~ exponential(1);
    vector[N] sigma = x + 1;
    y ~ normal(0, sigma);
}
"""

data = {
    'N': 3,
    'y': [1, 2, 3]
}
stan_model = stan.build(program_code=stan_model_code, data=data)

Building...



Building: 57.6s, done.

Before we compare the syntaxes, let's confirm they are equivalent, by checking the log_prob matches.

By default STAN drops normalizing constants out of densities, whereas PyMC does not, so we'll have to check that the delta between two points matches.

In [ ]:
x_log_test_value1 = [-1, 0, 1]
x_log_test_value2 = [1, 1, 2]

In [ ]:
res1 = stan_model.log_prob(x_log_test_value1)
res2 = stan_model.log_prob(x_log_test_value2)
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-7.499, res2=-13.824, res1-res2=6.325'

In [ ]:
pymc_log_prob = pymc_model.compile_logp()
res1 = pymc_log_prob({"x_log__": x_log_test_value1})
res2 = pymc_log_prob({"x_log__": x_log_test_value2})
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-10.255, res2=-16.581, res1-res2=6.325'

## Line by line comparison

### Data

The first Stan block defines the data by specifying it's a vector of length `N`, called `y`.

```stan
data {
  int< lower=0 > N;
  vector[N] y;
}
```

This is all defined implicitly in the PyMC model by specifying `observed = [1, 2, 3]`, but can be done explicitly by using a `Data` container:

```python
with pm.Model() as pymc_model:
  y_data = pm.Data("y_data", [1, 2, 3])
  ...
  pm.Normal("y", ..., observed=y_data)
```

Note we have to give a different name to the data container and the y variable. This is so we can access either by name later (`pymc_model["y"]` and `pymc_model["y_data"]`)

### Parameters

One of the largest differences between the two libraries, is that in Stan you have to explicitly define the model parameters and constraints, separately from how they "are distributed".

```
parameters {
  vector< lower=0 >[3] x;
}
```



In PyMC a parameter is implictly defined everytime a "named variable" is defined inside a model.

```ptyhon
x = Exponential("x", 1, shape=(3,))
```

Note the repetition of `x`. The Python variable name need not match the model  variable name, and you can also reassign it to other Python variables just like you would expect from python objects.

If you do not need to use `x`, you don't need to assign it to any python Variable, like we do with `y` later.


The variable assigned to `x` in PyMC is rather different in nature than the Stan one. It is not a "parameter", but a "random variable" that you can evaluate to take random draws.

In [ ]:
draw(x)

array([0.83493973, 0.50597248, 1.46389397])

The actual parameter is tucked away inside the model in `rvs_to_values`:

In [ ]:
pymc_model.rvs_to_values[x]

x_log__

Unlike Stan, the positive constrain tranform is automatically applied, because in regular applications of PyMC a parameter "belongs" to a specific distribution.

Because, an `Exponential` is a positive-domain distribution, PyMC applies a default log transform to the parameter `x`, that encodes the constraint and frees the samplers from boundary conditions.

The transform can be found in `rvs_to_transforms`

In [ ]:
pymc_model.rvs_to_transforms[x]

The constraint can be customized/disabled by passing an approriate value to `default_transform` when defining the random variable as in:

`x =  Exponential("x", default_transform=None)`

Both PyMC and Stan apply the jacobian correction of the transformation by default, so that the prior is respected when sampling from the unconstrained space. Both can also be opted out:

In [ ]:
res1 = stan_model.log_prob(x_log_test_value1, adjust_transform=False)
res2 = stan_model.log_prob(x_log_test_value2, adjust_transform=False)
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-7.499, res2=-17.824, res1-res2=10.325'

In [ ]:
pymc_log_prob = pymc_model.compile_logp(jacobian=False)
res1 = pymc_log_prob({"x_log__": x_log_test_value1})
res2 = pymc_log_prob({"x_log__": x_log_test_value2})
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-10.255, res2=-20.581, res1-res2=10.325'

### Intermediate variables

In Stan, if you explicitly assign intermediate variables, you have to define their type:

```stan
vector[N] sigma = x + 1;
```


In PyMC the type is inferred automatically
```python
sigma = x + 1
```

In [ ]:
sigma.type

TensorType(float64, shape=(3,))

Both Stan and PyMC allow you to save these intermediate variables during sampling.

For Stan you include the variable definition inside a `generated quantities` block.

For PyMC you wrap the variable in a `Deterministic` block (even though the variable need not be deterministic!).

### Likelihood

In Stan, after having specified `y` is data, we assign its density like we do with a regular parameter

```stan
y ~ normal(0, sigma);
```

In PyMC you pass the observed argument instead.

```python
pm.Normal("y", ..., observed=y_data)
```

Because we don't intend to use `y` anywhere else, we didn't assign it to a Python variable. We can still retrieve it, and we'll see that like `x` this is just another "random variable".

In [ ]:
y = pymc_model["y"]
draw(y)

array([ 5.42768049, -7.29585954,  2.58802198])

The only immediate sign of the `observed` argument is the shape is automatically derived from the observed data.

Under the hood, the variable is distinguished by being stored in a separate model property: `observed_RVs`, instead of `free_RVs`

In [ ]:
pymc_model.observed_RVs, pymc_model.free_RVs

([y], [x])

The `observed` constitutes the "value" of the `y` RV, which is a constant, not a parameter.

In [ ]:
pymc_model.rvs_to_values[y]

TensorConstant(TensorType(float64, shape=(3,)), data=array([1., 2., 3.]))

### Aside: observed can be deferred.

PyMC introduced a `pm.observe` model transformation, which allows one to define the observed quantities at a later time. In this case the shape (or dims) have to be provided explicitly.

In [ ]:
with Model() as raw_model:
    # Not assigning to Python variables to avoid shadowing the ones from the original example
    Exponential("x", 1, shape=(3,))
    Normal("y", 0, raw_model["x"] + 1, shape=(3,))

print(raw_model.free_RVs, raw_model.observed_RVs)

observed_model = observe(raw_model, {"y": [1, 2, 3]})

print(observed_model.free_RVs, observed_model.observed_RVs)

[x, y] []
[x] [y]


These shapes can be defined in a way that the lengths are not pre-commited, but the user must still pre-commit to the dimensionality (`ndim`) of the variable.

## Forward draws

When we define a PyMC model we are actually defining the random graph of the model, from which we can take draws by simply evaluating the nodes in a "forward sampling scheme".

In [ ]:
draw(x)

array([0.14709294, 0.29718136, 2.39086696])

In [ ]:
draw(y)

array([-1.21047051,  2.66030059, -2.09453266])

When calling `draw(y)` alone, new draws of `x` are implicitly drawn as well, just not returned.

One can request draws from multiple variables simultaneously to get the joint "consistent" forward samples of both random variables.

In [ ]:
draw([x, y])

[array([1.96750042, 0.44927411, 0.48087822]),
 array([ 0.84274034, -1.69644508,  1.58921521])]

PyMC does just this for prior predictive sampling, avoiding running a more expensive mcmc sampler/ worrying about convergence issues.

Similarly, for posterior predictive sampling, PyMC takes forward draws of the observed variables, but reuses the posterior draws of the free variables instead of evaluating them again.

In [ ]:
implausible_posterior_x_sample = np.array([1, 10, 100.])
draw(y, givens={x: implausible_posterior_x_sample})

array([ -1.58951369, -10.38902359, 166.01541509])

To perform forward sampling in Stan one must redefine the equivalent variables using random number generators in the `generated_quantities` block.

```stan
generated quantities {
    vector[N] y_sim;
    for (n in 1:N) {
        y_sim[n] = normal_rng(0, sigma);
    }
}
```

(Not sure if vectorized rng functions are provided or must still be done in a loop)

## The one parameter ↔ one distribution assumption

It's worth discussing this PyMC assumption a bit further, as it is where it deviates the most from Stan.

In Stan it's valid and (straightforward) to assign multiple densities to the same parameter, or assign widely different densities to different dimensions of the parameter.

Let's use a new example to illustrate this. It's almost the same as the previous model, except the first two entries of x are first assigned an Exponential density, and the last one a HalfNormal density.

Then all entries are assigned a second Normal density on top of the first one.

And a cringe mathematical-like definition:

$$
\begin{align*}
x_i &\sim \text{Exponential}(1) \quad \text{for } i = 1, 2 \\
x_3 &\sim \text{HalfNormal}(1) \\
x_i &\sim \text{Normal}(0, 1) \quad \text{for } i = 1, 2, 3 \\
y_i &\sim \text{Normal}(0, x_i + 1) \quad \text{for } i = 1, 2, 3 \\
\end{align*}
$$

In [ ]:
stan_model_code2 = """
data {
  int< lower=0 > N;\
  vector[N] y;
}

parameters {
  vector< lower=0 >[3] x;
}

model {
    x[1:2] ~ exponential(1);
    x[3] ~ normal(0, 1);
    x ~ normal(0, 1);
    y ~ normal(0, x + 1);
}
"""

data = {
    'N': 3,
    'y': [1, 2, 3]
}

stan_model2 = stan.build(program_code=stan_model_code2, data=data)

Building...



Building: found in cache, done.Messages from stanc:


Stan emits a warning saying the parameter has 3 priors (technically it only has 2 ^^)

Stan doesn't have a HalfNormal distribution, instead we get one implicitly by assiging a zero-centered Normal to a positively constrained parameter.

PyMC doesn't really have an equivalent version of this model, at least not the sort of PyMC usage that you're likely to find in the wild. But we can achieve the same if we really want to.

I'll show two approaches, to give a better picture.

### CustomDist for crazy distributions

For the first two densities we can stay in PyMC-approved land, by using CustomDist.

In [ ]:
with Model() as pymc_model2:
    def first_two_exponential_third_normal_dist(_):
        return math.concatenate(
            [Exponential.dist(1, shape=(2,),), HalfNormal.dist(1, shape=(1,))]
        )

    x = CustomDist(
        "x",
        dist=first_two_exponential_third_normal_dist,
        transform=transforms.log,
    )
    y = Normal("y", 0, x + 1, observed=[0, 1, 2])

`CustomDist` accepts a callable as the `dist` argument, that defines an implicit distribution from a generative random graph. In this case the concatenation of two Exponential random variables and one HalfNormal random variable.

We have to use the `.dist` to define floating random variables. These are not model variables and have no parameters, they exist just to define the graph that represents the distribution we care about. The actual random variable and parameters are the `"x"` defined by the `CustomDist`.

We still live in the land of one parameter, one distribution, just a "custom" one. Because it was defined via the random graph, PyMC has no trouble taking draws from it:

In [ ]:
draw(x)

array([0.99874554, 0.87974753, 0.74318369])

We no longer get default transforms, they have to be defined explicitly. I'm purposedly using the argument `transform` instead of `default_transform` here, to highlight this difference.

**Note that the transforms don't have any effect in the random graph. Had we used a Normal instead of a HalfNormal, we would see negative values on the third entry half of the time.**

PyMC can also get the log_prob of our `CustomDist`, by reasoning symbolically about the random forward graph implied in the `dist` function. In this case it's a trivial question of pairing the right entries of the parameter with the respective densities, but it can be richer than this.

If you are curious about what sorts of densities PyMC can infer from random graphs, check out [30 short puzzles on probability](https://colab.research.google.com/github/ricardoV94/probability-puzzles/blob/main/puzzles.ipynb)

In [ ]:
pymc_model2.compile_logp()({"x_log__": [-1, 0, 1]})

array(-10.63434397)

### Breaking free with Potentials

For the second density assignment we have to break free with our dream of having automatically matching random and log_prob graphs under a single PyMC model.

PyMC provides `Potential` to define arbitrary terms that should be added to the model joint logp, and which have no random counterpart.

In [ ]:
with pymc_model2:
    Potential(
        "non_default_density_on_x",
        Normal.logp(x, 0, 1)
    )

(Annoyingly you have to give them unique names...)

Now the two models should be equivalent

In [ ]:
res1 = stan_model2.log_prob(x_log_test_value1)
res2 = stan_model2.log_prob(x_log_test_value2)
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-12.737, res2=-68.422, res1-res2=55.685'

In [ ]:
pymc_log_prob = pymc_model2.compile_logp()
res1 = pymc_log_prob({"x_log__": x_log_test_value1})
res2 = pymc_log_prob({"x_log__": x_log_test_value2})
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-17.653, res2=-73.981, res1-res2=56.328'

(It's not exactly the same though, so I may be missing something small, or we see rounding errors here)

And if you try to use a forward sampling routine like `sample_prior_predictive` or `sample_posterior_predictive` PyMC will warn you that Potential terms are simply ignored by those.

In [ ]:
with pymc_model2:
    sample_prior_predictive()

<ipython-input-30-72f58f10cd3e>:2: UserWarning: The effect of Potentials on other parameters is ignored during prior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  sample_prior_predictive()


We want to add a [second warning](https://github.com/pymc-devs/pymc/discussions/5404) for when a user defines a `transform` (not a `default_transform`), for similar reasons. Custom transforms don't have any effect on forward draws, and can cause a mismatch between the forward and logp sampling methods of the same model.

`draw` has no concept of a Model, and simply evaluates the random graph without any warnings.

In [ ]:
draw(x)

array([0.37580074, 0.11988222, 0.37292105])

## Writing PyMC models as if it were Stan

It is possible, although not pretty, and likely silly, to translate Stan models to PyMC line-by-line.

The trick is to force PyMC to create parameters without assigned-densities, using `Flat`, and providing custom `transform`s to define the constraints on these parameters.

Once we have the variable returned by `Flat` we can chain arbitrary Potential terms like Stan does once you strip the syntactic sugar.

In [ ]:
with Model() as pymc_model2_eq:
    x = Flat("x", shape=(3,), default_transform=transforms.log)

    Potential("x_term[1:2]", Exponential.logp(x[:2], 1))
    Potential("x_term[3]", Normal.logp(x[2], 0, 1))
    Potential("x_term", Normal.logp(x, 0, 1))
    Potential("y_term", Normal.logp(np.array([0, 1, 2]), 0, x + 1))

In [ ]:
pymc_log_prob = pymc_model2_eq.compile_logp()
res1 = pymc_log_prob({"x_log__": x_log_test_value1})
res2 = pymc_log_prob({"x_log__": x_log_test_value2})
f"{res1=:.3f}, {res2=:.3f}, {res1-res2=:.3f}"

'res1=-18.347, res2=-74.674, res1-res2=56.328'

### Note on requesting densities from PyMC

Final note, for brevity I used `Normal.logp(x, 0, 1)` syntax which is discouraged, as not all Distributions have a `.logp` method and the parametrization accepted by them can be non-intuitive. The "valid" way to get the logp would be:

```python
from pymc import logp

logp(Normal.dist(0, 1), x)
```
Where we create a dummy `Normal` random variate just to extract the corresponding logp term. This will work with any distribution / parametrization, as well as arbitrary graphs of random variables, as long as PyMC can derive its density.